# ccdproc-01:   Classify images  

## Overview
1. Start CCD reduction using ccdproc
2. Creating lists of images 
3. Examples

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano and Nicolás Cardiel

This notebook has reproduced parts of the the astropy ccdproc docs: 
https://ccdproc.readthedocs.io/en/latest/image_management.html

Version 1.0 2021/01/11  
</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.

In [1]:
from pathlib import Path
from astropy.io import fits
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
plt.style.use('./tea.mplstyle')

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the directory with the downloaded images

In [3]:
directory = 'N1'
dirname = 'N1'

In [4]:
# Version simple
# solo descarga si es necesario
#!curl -C -  -O ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/NOT_20080412_N1.tar.gz

In [5]:
# Version simple
#!mkdir -p N1
#!tar axf NOT_20080412_N1.tar.gz --strip-components=1 -C N1
# o bien
#!tar -x -f NOT_20080412_N1.tar.gz -a --strip-components=1 -C $directory

In [6]:
%%bash
# Version complicada, solo descarga si es necesario
TARBALL=NOT_20080412_N1.tar.gz
DIR=N1
URI=ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/"$TARBALL"
if [ ! -f "$TARBALL" ]; then
    echo "$TARBALL does not exist, download"
    curl -C -  -O "$URI"
fi
if  [ ! -d "$DIR" ]; then
    mkdir -p $DIR
    tar -x -f NOT_20080412_N1.tar.gz -a --strip-components=1 -C $DIR
fi

Let\'s create a list containing all the FITS files in that directory 

In [7]:
# only works in notebooks
# we can pass the python variable to shell
filelist = !ls $directory/AL*.fits

In [8]:

p = Path(directory)
filelist = list(sorted(p.glob('AL*.fits')))
print(filelist[10:20])    # printing only from 10 to 20

[PosixPath('N1/AL12011.fits'), PosixPath('N1/AL12012.fits'), PosixPath('N1/AL12013.fits'), PosixPath('N1/AL12014.fits'), PosixPath('N1/AL12015.fits'), PosixPath('N1/AL12016.fits'), PosixPath('N1/AL12017.fits'), PosixPath('N1/AL12018.fits'), PosixPath('N1/AL12019.fits'), PosixPath('N1/AL12020.fits')]


Let\'s open one of the files to explore the HEADER keywords

In [ ]:
HDUList_object = fits.open(filelist[102])
HDUList_object.info()

In [ ]:
primary_HDU = HDUList_object[0]

In [ ]:
# Extract primary header
primary_header = primary_HDU.header

# Index header object with keyword name and print value
print(primary_header['FILENAME'],primary_header['OBJECT'])

In [ ]:
for key in primary_header:
    print('{:8} = {}'.format(key, primary_header[key]))

Each telescope/instrument has differences in the ``keyword``names.  
For [ALFOSC at NOT](http://www.not.iac.es/instruments/alfosc/) instead of ``FILTER`` they write ``ALFLTID`` and  ``FBFLTID``  in the header. ALFOSC has  filterwheel A and filterwheel B and the filters should be mounted in one of the two depending of size and/or band width. More info at [NOT filters](http://www.not.iac.es/instruments/filters/)

Let\'s display this file

In [ ]:
image_data = primary_HDU.data
print(type(image_data))
print(image_data.shape)

In [ ]:
naxis2, naxis1 = image_data.shape

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
img = ax.imshow(image_data, vmin=5000, vmax=8000)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')
ax.grid()

In [ ]:
HDUList_object.close()

This is the first of three 300s exposures of NGC4496 with the R filter.    
The external parts of the chip (2052 x 2198) are black, i.e. no signal. according with the header information the region reserved for dark signal is BIASSEC  = [3:52,1:2052] but there is also a region at right. We will remove these regions by trimming the image. It is better to use ``DS9`` to select the region of interest. 

### Looking for files with different size

There are some auxiliary observations that were saved as FITS files. For instance those for focusing the telescope. There is no problem in processing them but there will be problems if there is an image with different size.

In [ ]:
for fname in filelist:
    with fits.open(fname) as HDUList_object:
        primary_header = HDUList_object[0].header
        print(primary_header['FILENAME'],primary_header['OBJECT'],primary_header['NAXIS1'],primary_header['NAXIS2'])

The image number 63 is different with size 800,800. It is better to remove it to avoid problems during the processing.

 * ALrd120062.fits Sky Flat evening 2198 2052
 * ALrd120063.fits Sky Flat evening 800 800
 * ALrd120064.fits HZ44 focusing 2198 2052

In [ ]:
!rm -f $dirname/ALrd120063.fits

### Selecting files by type and filter  
We wish to create lists containing the BIAS files to build the master BIAS or those scientific files taken with the same filter to apply the corresponding Flat Field, to name two examples.

In [ ]:
from ccdproc import ImageFileCollection
from ccdproc.utils.sample_directory import sample_directory_with_files
directory = 'N1'
ic_all = ImageFileCollection(directory, keywords='*')
print(ic_all.summary.colnames)

Selecting the keywords to be used in classifying the files  
ALFLTID =                   76 / ALFOSC filter wheel. NOT filter number  
FAFLTID =                    0 / FASU filter wheel 1/A NOT filter number        FBFLTID =                    0 / FASU filter wheel 2/B NOT filter number   

In [ ]:
keys = ['OBJECT' , 'EXPTIME' , 'ALFLTID' , 'FAFLTID' , 'FBFLTID']
ic1 = ImageFileCollection(directory, keywords=keys) # only keep track of keys
ic1.summary.colnames

In [ ]:
# matches = (ic1.summary['filter'] == 'R') & (ic1.summary['exposure'] < 15)
#matches = (ic1.summary['imagetyp'] == 'BIAS') 
#matches = (ic1.summary['FBFLTID'] == 78)
matches = (ic1.summary['EXPTIME'] > 600)
my_files = ic1.summary['file'][matches]
print(my_files)

In [ ]:
my_files = ic1.files_filtered(FBFLTID=78, exptime=3.5)
#my_files = ic1.files_filtered(exptime=15)
print(my_files)

In [ ]:
bias_list = ic1.files_filtered(regex_match=True,imagetyp='bias|light')
print(bias_list)

In [ ]:
list_of_flats = ic1.files_filtered(regex_match=True,object='flat')
print(list_of_flats)